In [1]:
import os
os.chdir("..")

import sim
from sim.games import pitch, play_game
from sim import params, Player, Team, League

params.ITERATIONS = 10_000

# About the sim
This is a very simple baseball sim, meant to express the most basic stats.

*Note: all constants are defined in params. This example uses the default values.*

## About Players

Every player has four attributes. These are randomly generated and typically range from 0 - 1. 
These attributes combine into a single weighted attribute, based on the attribute weights.
The first attribute makes up 60% of a player's effectiveness; the second and third are 30% and 10% respectively.
The fourth attribute is entirely worthless.

This represents more and less important attributes in the real MMOLB sim.

In [2]:
print("Attribute weights:")
print(params.STAT_WEIGHTS)
print()

print("Example player:")
p = Player(0)
print(p.attributes)
print(f"This player has a total weighted attribute of {p.weighted}")

Attribute weights:
[0.6, 0.3, 0.1, 0.0]

Example player:
[0.7598520286720064, 0.48246864741952566, 0.9665920304469848, 0.21969827720028534]
This player has a total weighted attribute of 0.69731101447376


Like real baseball (and also real fake baseball), this sim consists of a series of duels between a pitcher and a batter.
However, unlike real baseball, which suffers from an entire three true outcomes, here there are exactly two outcomes:
either the player strikes out, or they hit a home run.

To determine which happens, the batter's weighted attribute is compared to the pitcher's weighted attribute.
If the players are equal, the batter has a 0.200 hit rate. The higher the batter's weighted attribute is vs. the pitcher, the higher the
hit rate.

In [3]:
good_player = Player(0)
good_player.attributes = [0.9, 0.6, 0.3, 0.1]
print(f"Good player's weighted attribute is {good_player.weighted:.2f}.")

bad_player = Player(0)
bad_player.attributes = [0.1, 0.3, 0.6, 0.9]
print(f"Bad player's weighted attribute is {bad_player.weighted:.2f}.")

average_pitcher = Player(1)
average_pitcher.attributes = [0.5] * 4
print(f"Average pitcher's weighted attribute is {average_pitcher.weighted:.2f}.")

Good player's weighted attribute is 0.75.
Bad player's weighted attribute is 0.21.
Average pitcher's weighted attribute is 0.50.


In [4]:
for __ in range(params.ITERATIONS):
    pitch(average_pitcher, good_player)
    pitch(average_pitcher, bad_player)

good_ba = good_player.hits/good_player.at_bats
bad_ba = bad_player.hits/bad_player.at_bats

print(f"After {params.ITERATIONS} iterations:")
print(f"Good player has {good_player.hits} hits and home runs, with a batting average of {good_ba:.3f} and a slugging of {good_ba * 4:.3f}.")
print(f"Bad player has {bad_player.hits} hits and home runs, with a batting average of {bad_ba:.3f} and a slugging of {bad_ba * 4:.3f}.")
print("Arron Judge slugged 0.701 last year so this sim is a lil dumb.")

After 10000 iterations:
Good player has 2382 hits and home runs, with a batting average of 0.238 and a slugging of 0.953.
Bad player has 1505 hits and home runs, with a batting average of 0.150 and a slugging of 0.602.
Arron Judge slugged 0.701 last year so this sim is a lil dumb.


# About Games and Teams

Just like baseball, games are made up of nine innings. Each half-inning plays until three outs are achieved.
For simplicity, games play both sides of the ninth inning (shame!) and if games are tied after nine innings wins are determined by coin flip.

In [5]:
good_team = Team(1, good_player.weighted, good_player.weighted)
bad_team = Team(2, bad_player.weighted, bad_player.weighted)
average_team = Team(3, average_pitcher.weighted, average_pitcher.weighted)

In [6]:
for __ in range(params.ITERATIONS):
    play_game(average_team, good_team)
    play_game(average_team, bad_team)

print(f"After {params.ITERATIONS} games against a perfectly average team:")
print(
    f"A team made up of entirely good players has won {good_team.wins}, for a win rate of {good_team.wins/good_team.games*100:.2f}%, "
    f"with an average run differential of {(good_team.runs - good_team.runs_surrendered)/good_team.games:.3f}."
)
print(
    f"A team made up of entirely bad players has won {bad_team.wins}, for a win rate of {bad_team.wins/bad_team.games*100:.2f}%, "
    f"with an average run differential of {(bad_team.runs - bad_team.runs_surrendered)/bad_team.games:.3f}."
)
print("Note that in this example notebook, bad player is more bad than good player is good.")

After 10000 games against a perfectly average team:
A team made up of entirely good players has won 7180, for a win rate of 71.80%, with an average run differential of 2.731.
A team made up of entirely bad players has won 2164, for a win rate of 21.64%, with an average run differential of -3.563.
Note that in this example notebook, bad player is more bad than good player is good.


## About Leagues and matchmaking

Each league has 100 teams, which play each other 100 times in a season.

Matchmaking works as follows:
First, the teams are sorted by total wins (and randomly otherwise). 
Then, going down the list in order, each team is matched with the following process:
1. If the team is already matchmade, continue down the list.
2. All teams within params.MATCHMAKING_SPREAD are selected.
3. Any teams in that window which are matchmade are filtered out.
4. One of the remaining teams is selected at random to play the current team.

This is not random - teams are more likely to match with teams at the edge of the window - but it's close enough.

In [7]:
league = League()
league.run_season()

team_stats, player_stats = league.to_dataframes()
display(team_stats.sort_values("wins", ascending=False))
display(player_stats.sort_values("attr_0", ascending=False))

,base_min,base_max,good_min,good_max,games,wins,total_runs,total_runs_surrendered,p0_team,p0_scale,...,b3_is_pitcher,b4_team,b4_scale,b4_attr_0,b4_attr_1,b4_attr_2,b4_attr_3,b4_at_bats,b4_hits,b4_is_pitcher
32,0,1,0,1,100,62,766,669,33,0.5,...,True,33,0.5,0.373221,0.826859,0.438550,0.810604,687,147,True
47,0,1,0,1,100,61,779,684,48,0.5,...,True,48,0.5,0.685562,0.311343,0.147431,0.706890,685,145,True
41,0,1,0,1,100,60,719,640,42,0.5,...,True,42,0.5,0.717283,0.033470,0.940582,0.708688,672,132,True
4,0,1,0,1,100,59,741,694,5,0.5,...,True,5,0.5,0.855557,0.177317,0.802409,0.854119,676,136,True
76,0,1,0,1,100,59,775,677,77,0.5,...,True,77,0.5,0.562376,0.959690,0.850588,0.684125,658,118,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,0,1,0,1,100,42,691,808,37,0.5,...,True,37,0.5,0.000904,0.736387,0.073105,0.634238,740,200,True
58,0,1,0,1,100,40,758,864,59,0.5,...,True,59,0.5,0.501907,0.055121,0.134426,0.294626,715,175,True
0,0,1,0,1,100,39,658,777,1,0.5,...,True,1,0.5,0.666402,0.774511,0.784711,0.954978,648,108,True
97,0,1,0,1,100,39,690,783,98,0.5,...,True,98,0.5,0.021837,0.712493,0.791529,0.028937,718,178,True


,team,scale,attr_0,attr_1,attr_2,attr_3,at_bats,hits,is_pitcher
373,25,0.5,0.998997,0.428280,0.366949,0.429219,316,86,False
855,58,0.5,0.998913,0.930303,0.404848,0.202682,639,99,False
1146,77,0.5,0.998842,0.830446,0.701817,0.690481,396,103,False
1091,73,0.5,0.998092,0.775112,0.337962,0.211650,336,99,False
646,44,0.5,0.997927,0.479206,0.086828,0.060232,664,124,True
...,...,...,...,...,...,...,...,...,...
1020,69,0.5,0.001403,0.693419,0.414584,0.160539,726,186,False
242,17,0.5,0.000949,0.373301,0.659883,0.422981,727,187,True
544,37,0.5,0.000904,0.736387,0.073105,0.634238,740,200,True
1034,69,0.5,0.000567,0.125253,0.720729,0.047330,307,38,False
